<a href="https://colab.research.google.com/github/himanshuRepo/CoLab_Notebooks/blob/master/CNN_in_TensorFlow_on_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Network on MNIST Dataset 

##Mount the Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Check the working directory

In [0]:
!ls "/content/drive/My Drive/"

##Change to a specific folder on Google Drive

In [0]:
import os
os.chdir("drive/My Drive/")

## Displaying the considered CNN

In [0]:
from IPython.display import Image
Image('./images/02_network_flowchart.png')

## Importing the libraries

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [0]:
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
tbc=TensorBoardColab()

# Load Dataset

In [0]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

In [0]:
# Get the first images from the test-set.
images = mnist.test.images[0:9]
plt.imshow(images[0].reshape(28,28), cmap='gray')
plt.axis('off')
plt.show()

# Variable Declaration

In [0]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [0]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

# TensorFlow Operation for Convolution and Pooling Layer

In [0]:
# Create the TensorFlow operation for convolution.
    # Note the strides are set to 1 in all dimensions.
    # The first and last stride must always be 1, because the first is for 
    # the image-number and the last is for the input-channel.
    # e.g. strides=[1, 2, 2, 1] would mean that the filter is moved 2 pixels 
    # across the x- and y-axis of the image.
    # The padding is set to 'SAME' which means the input image
    # is padded with zeroes so the size of the output is the same.
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

# First Convolution and Pooling Layer

In [0]:
#Its weight tensor will have a shape of [5, 5, 1, 32]. 
#The first two dimensions are the patch size, the next is the number of input channels, 
#and the last is the number of output channels. 
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

In [0]:
#First dim means for all. the second and third dimensions corresponding to image width and height, 
#and the final dimension corresponding to the number of color channels.
x_image = tf.reshape(x, [-1, 28, 28, 1])

In [0]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# Second Convolution and Pooling Layer

In [0]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# Fully Connected, Pooling, and Dropout Layer

In [0]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [0]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [0]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

# Cross Entropy

In [0]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Session Code

In [0]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(200):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
      train_accuracy = accuracy.eval(feed_dict={
          x: batch[0], y_: batch[1], keep_prob: 1.0})
      print('step %d, training accuracy %g' % (i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

  print('test accuracy %g' % accuracy.eval(feed_dict={
      x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))
  summary_writer = tbc.get_writer()
  summary_writer.add_graph(sess.graph)
